In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import time


In [5]:
df = pd.read_csv('data\kospi200_company_code.csv')
df


<>:1: SyntaxWarning: invalid escape sequence '\k'
<>:1: SyntaxWarning: invalid escape sequence '\k'
C:\Users\kiwoom\AppData\Local\Temp\ipykernel_28340\4139707292.py:1: SyntaxWarning: invalid escape sequence '\k'
  df = pd.read_csv('data\kospi200_company_code.csv')


,company_name,stock_code
0,한화비전,489790
1,HD현대마린솔루션,443060
2,에이피알,278470
3,엘앤에프,66970
4,포스코DX,22100
...,...,...
195,대한전선,1440
196,한국앤컴퍼니,240
197,대한항공,3490
198,유한양행,100


In [11]:
# 🔹 티커 변환 함수
def to_yf_ticker(stock_code):
    return f"{str(stock_code).zfill(6)}.KS"

# 🔹 기간 설정 (최근 1년)
end_date = pd.Timestamp.today()
start_date = end_date - pd.DateOffset(years=1)

# 🔹 결과 저장 컬럼 초기화
df['vol_1y'] = np.nan
df['missing_count'] = np.nan

# 🔹 실패한 종목 저장 리스트
failed_tickers = []

# 🔁 종목별 계산
for idx, row in df.iterrows():
    company = row['company_name']
    code = row['stock_code']
    ticker = to_yf_ticker(code)

    try:
        # ▷ 주가 다운로드 (종가만)
        price = yf.download(ticker, start=start_date.strftime('%Y-%m-%d'),
                            end=end_date.strftime('%Y-%m-%d'),
                            progress=False)[['Close']]
        price.rename(columns={'Close': company}, inplace=True)

        # ▷ 결측치 개수 확인
        missing_count = price[company].isna().sum()
        df.at[idx, 'missing_count'] = missing_count

        # ▷ 결측치 처리
        price = price.ffill().bfill()

        # ▷ 로그수익률 계산
        log_ret = np.log(price / price.shift(1)).dropna()

        # ▷ 변동성 계산 (연환산)
        if not log_ret.empty:
            vol = log_ret.std().values[0] * np.sqrt(252)
            df.at[idx, 'vol_1y'] = vol

        time.sleep(0.3)

    except Exception as e:
        failed_tickers.append((company, ticker))

# ✅ 결과 출력
print("✅ 1년치 변동성 계산 완료")
print(df[['company_name', 'stock_code', 'vol_1y', 'missing_count']])

if failed_tickers:
    print("\n❌ 실패한 종목:")
    for name, ticker in failed_tickers:
        print(f"- {name} ({ticker})")

C:\Users\kiwoom\AppData\Local\Temp\ipykernel_28340\3512471313.py:36: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  df.at[idx, 'missing_count'] = missing_count
C:\Users\kiwoom\AppData\Local\Temp\ipykernel_28340\3512471313.py:36: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  df.at[idx, 'missing_count'] = missing_count
C:\Users\kiwoom\AppData\Local\Temp\ipykernel_28340\3512471313.py:36: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  df.at[idx, 'missing_count'] = missing_count
C:\Users\kiwoom\AppData\Local\Temp\ipykernel_28340\3512471313.py:36: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  df.at[idx, '

✅ 1년치 변동성 계산 완료
    company_name  stock_code    vol_1y  missing_count
0           한화비전      489790  0.853230            0.0
1      HD현대마린솔루션      443060  0.583690            0.0
2           에이피알      278470  0.643910            0.0
3           엘앤에프       66970  0.700304            0.0
4          포스코DX       22100  0.615264            0.0
..           ...         ...       ...            ...
195         대한전선        1440  0.478358            0.0
196       한국앤컴퍼니         240  0.386239            0.0
197         대한항공        3490  0.286512            0.0
198         유한양행         100  0.541250            0.0
199       CJ대한통운         120  0.337060            0.0

[200 rows x 4 columns]


## --> 기업별로 1년치 변동성 구하기